In [36]:
# Load SciPy and its optmization package
import scipy
from scipy import optimize
# Load a list of integrals of Laguerre Gaussians
Lag=load('laguerre_integrals')


##########################################################################################################
# -----------------------------------------------------------------
#  sys_verify
# -----------------------------------------------------------------
#
# INPUT:
# - genus: the genus in which we want to prove a bound
# - param: a pair (Z,W), where:
#          * Z is a list of prescribed double roots for f(sqrt(x))
#          * W is a list of prescribed double roots for f^(sqrt(x))
#
# THE PROGRAM:
# - Computes a Fourier pair (f,f^) based on "genus" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of parameters: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "genus" and "param"    
# - Verifies whether the pair (f,f^) satisfies the conditions of Theorem 6.1
# - If so, gives a rigorous estimate for the bound from that Theorem
#
# OUTPUT:
# - If the conditions are satisfied: the computed bound
# - If not, an error message
##########################################################################################################

def sys_verify(genus,param):
    print('\n\n----------------------------------------------------------------')
    print('sys_verify')
    print('genus: ', genus, '\n\n')
    
    (Z,W) = param
    
    # Make sure the entries of Z an W are treated as rational numbers
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]
    
    m = len(Z)
    n = len(W)
    
    # In order to be able to count the zeroes later on, we restrict the intervals in which they're
    # allowed to lie
    for elem in Z:
        if elem<=0:
            return 'The forced double zeros for u should be strictly positive'
    for elem in W:
        if elem<=-1/4:
            return 'The forced double zeros for v should be strictly greater than -1/4'
    
    # For convenience, we also assume that the zeros are listed in order:
    for j in range(1,m):
        if Z[j]<=Z[j-1]:
            return 'The list of forced double zeros for u is not increasing'
    for j in range(1,n):
        if W[j]<=W[j-1]:
            return 'The list of forced double zeros for v is not increasing'
    
    # Compute the polynomials u and v
    (u,v) = sys_poly(genus,param)
    u_der = u.derivative()
    u_der_der = u_der.derivative()
    v_der = v.derivative()
    v_der_der = v_der.derivative()
    
    print('Computed the pair of polynomials (u,v) based on the input')
    
    # Check part of the sign conditions from Theorem 6.1
    if u(0)<0: # Check that f is non-negative at the origin
        return 'f is negative at the origin'
        
    elif sgn(u.leading_coefficient())==1:   # Check that f is eventually negative
        return 'f takes positive values near infinity'

    elif sgn(v.leading_coefficient())==-1:  # Check that f^ is eventually positive
        return 'f^ takes negative values near infinity'  

    
    print('Verified the first conditions on the signs of (u,v)')

    
    # Our next step is to estimate the single root of u.
    # This estimate will a priori not be rigorous.
    # However, we will a posteriori rigorously verify that u satisfies the necessary conditions
    
    a=0
    if m!=0:
        b = Z[0]-0.001  # We start a bit to the left of the first double zero of u
            
    else:  # If we are not imposing any double zeros, start far enough 
        b = genus 
        while u(b)>0:
            b = 2*b 
        
    while u(b)>0: # If somehow we move too far left, we shift the interval to the right
        a=b
        b=b+0.001                    
            
    R = scipy.optimize.brentq(u,a,b)  # This finds an approximate root of u in [a,b]
    r = QQ(sqrt(R)) #R is the root of u and sqrt(R) is the root of f, which we turn into a rational number
    
    while u(r^2)>=0: # Make sure that u is negative at R
        r = r+1/10^8
    
    print('Found an approximation for the first root of u')
    
    # Check that r lies before the other roots of u:
    if m != 0 :
        if r^2 >= Z[0]:
            return 'Error in the root approximation: the first root we found for f lies past the prescribed roots'
    
    # Make sure that f and f^ satsify all the remaining conditions from Theorem 6.1
    if pari.polsturm(u,[r^2,Infinity])>m: # Check that past r^2, u does not have more real roots than prescribed
        return 'f has an extra zero'
                
    if pari.polsturm(v,[-1/4,Infinity])>n: # Check that v does not have more real roots than prescribed
        return 'f^ has an extra zero'
            
    # Check that u and v vanish to the right degree at the prescribed roots
    for j in range(m):
        if u(Z[j])!=0 :
            return 'f does not vanish at one of the prescribed zeros'
                        
        if u_der(Z[j])!=0 :
            return 'One of the prescribed zeros of f is not double'  

        if u_der_der(Z[j])>=0 :
            return 'The second derivative of f at one of the double roots is non-negative'

                
    for j in range(n):
        if v(W[j])!=0 :
            return 'f^ does not vanish at one of the prescribed zeros'
                        
        if v_der(W[j])!=0 :
            return 'One of the prescribed zeros of f^ is not double'     

        if v_der_der(W[j])<=0 :
            return 'The second derivative of f^ at one of the double roots is non-positive'
    
    print('Verified the other sign conditions on f and f^')
    
    # Compute an estimate for the integral
    # Below, RBF and CBF denote the RealBallField and ComplexBallField from the Arb package
    
    # h is defined so that f^(x) = h(x^2)
    def h(t):
        return v(t)*e^(-t/2)
    
    # We estimate the integral term in the Selberg trace formula
    truncated_integral = 2*(genus-1)*CBF.integral(lambda x, _: h(x^2)*x*tanh(CBF(pi)*x),0,200)
    
    print('Computed the truncated integral')
    
    # For the remainder of the integral, we compute an indefinite integral
    # We will use that x^(2*n+1) * exp(-x^2 / 2) admits a primitive
    # of the form p_n(x)*exp(-x^2 / 2), where p_n is a polynomial
    
    # First we compute the p_n's:
    max_d = v.degree() # the maximal degree for which we need to know p_n
    p = [-1] # initializing p[0] (which is p_1)
    for n in range(1,max_d):
        p += [-x^(2*n)+2*n*p[n-1]] # adding p_n to the list, the recurrence we use is obtained
                                   # from integration by parts
    
    # Compute the primitive of x*v(x^2)*exp(-x^2/2)
    V = 0
    for n in range(max_d):
        V += v.coefficients()[n]*p[n]
        
    remainder = -2*(genus-1)*V.subs(x==200)*exp(-200^2/2)
    
    print('Computed the remainder term of the integral')
    
    # Verify that f^(i/2) >= 2*(genus-1)*integral( f^(y)*y*tanh(pi*y) dy, 0, infinity )
    if not RBF(h(-1/4)) >= RBF(truncated_integral) + RBF(remainder):
        return 'The integral inequality is not satisfied'
    
    print('Verified the integral condition on f^')
    
    print('\n\nSystole bound: ',r)
    print('Decimal approximation: ',RDF(r))

    return r

##########################################################################################################
# -----------------------------------------------------------------
#  sys_poly
# -----------------------------------------------------------------
# INPUT:
# - genus: the genus in which we want to prove a bound
# - param: a pair (Z,W), where:
#          * Z is a set of prescribed double roots for f
#          * W is a set of prescribed double roots for f^
# 
# THE PROGRAM:
# - Computes a pair (u,v) of polynomials based on "genus" and "param":
#    f(x) will be of the form u(x^2)*e^(-x^2/2) for some polynomial u to be determined
#    f^(y) is v(y^2)*e^(-y^2/2) for some polynomial v to be determined
#    the degree d of u and v are determined by the total number of roots: it's the minimal degree such
#    that there is a pair of non-zero polynomials with the prescribed values. This implies that the pair
#    (u,v) is uniquely determined by "genus" and "param"    
#
# OUTPUT:
# (u,v)
##########################################################################################################

def sys_poly(genus,param):
    
    
    (Z,W) = param
    
    # Make sure the entries of Z an W are treated as rational numbers
    Z = [QQ(elem) for elem in Z]
    W = [QQ(elem) for elem in W]
    
    m = len(Z) # number of forced double roots for u
    n = len(W) # number of forced double roots for v
    c = 2+2*(m+n) # number of conditions imposed
    
    # Compile a list of Laguerre polynomials and their derivatices
    P.<x> = PolynomialRing(QQ) 
    L = [ P(gen_laguerre(n,-1/2,x)) for n in range(c)]
    L_der = [ L[n].derivative() for n in range(c)]
    
    # Load the integrals of the Laguerre Gaussians that we computed beforehand.
    # The correctness of these integrals does not need to be verifiable, they are only used to set linear
    # condition on v and don't influence whether or not u(x^2)*exp(-x^2/2) and v(y^2)*exp(-y^2)/2
    # are each other's Fourier transforms.
    # The fact that the pair (u,v) satisfies all the necessary conditions is verified a posteriori in
    # sys_verify.
    I = Lag[0:c]
    
    A = [[L[j](0) for j in range(c)]]            
    B = [[L[j](Z[k]) for j in range(c)] for k in range(m)] #for u to vanigh at the z_k
    C = [[L_der[j](Z[k]) for j in range(c)] for k in range(m)] #for u' to vanish at the z_k
    
    # We want to set up our functions so that the integral term in the Selberg trace formula
    # (essentially) equals f^(i/2). We need to add in a small error in order to absorb 
    # bounds on approximation errors before. The errors below are ad hoc and we found based 
    # on trial and error.
    if genus <= 15:
        err = 0.0001
    elif genus <=19:
        err = 0.001
    else:
        err = 0.005
        
    D = [[(-1)^j*(L[j](-1/4)*RDF(e^(1/8))-(1+err)*2*(genus-1)*I[j]) for j in range(c)]] # for f^(i/2) to (nearly) equal the integral term in the trace fomula
    E = [[(-1)^j*L[j](W[k]) for j in range(c)] for k in range(n)] # for v to vanish at the w_k
    F = [[(-1)^j*L_der[j](W[k]) for j in range(c)] for k in range(n)] # for v' to vanish at the w_k

    M = pari(matrix(QQ,A+B+C+D+E+F))
    Y = pari(matrix(QQ,[1]+[0]*(c-1)).transpose())
    X = vector(pari.matinverseimage(M,Y).sage())

    u = sum([X[j]*L[j] for j in range(c)]) # the polynomial for f 
    v = sum([(-1)^j*X[j]*L[j] for j in range(c)]) # the polynomial for f^
    
    return (u,v)

In [37]:
# Verify Table 1

# Load the systole parameters
sys_parameters = load('parameters_systole')

# Compile a list of systole bounds in genus 2 - 20
sys_list = []
for genus in range(2,21):
    param = sys_parameters[genus-2]
    sys_list += [RDF(sys_verify(genus,param))]

sys_list



----------------------------------------------------------------
sys_verify
genus:  2 


Computed the pair of polynomials (u,v) based on the input
Verified the first conditions on the signs of (u,v)
Found an approximation for the first root of u
Verified the other sign conditions on f and f^
Computed the truncated integral
Computed the remainder term of the integral
Verified the integral condition on f^


Systole bound:  127957040705433847/40543384700000000
Decimal approximation:  3.156052254942441


----------------------------------------------------------------
sys_verify
genus:  3 


Computed the pair of polynomials (u,v) based on the input
Verified the first conditions on the signs of (u,v)
Found an approximation for the first root of u
Verified the other sign conditions on f and f^
Computed the truncated integral
Computed the remainder term of the integral
Verified the integral condition on f^


Systole bound:  20550398748991127/4899112700000000
Decimal approximation:  4.194718

Computed the pair of polynomials (u,v) based on the input
Verified the first conditions on the signs of (u,v)
Found an approximation for the first root of u
Verified the other sign conditions on f and f^
Computed the truncated integral
Computed the remainder term of the integral
Verified the integral condition on f^


Systole bound:  1117052453/135882731
Decimal approximation:  8.22070946601743


----------------------------------------------------------------
sys_verify
genus:  20 


Computed the pair of polynomials (u,v) based on the input
Verified the first conditions on the signs of (u,v)
Found an approximation for the first root of u
Verified the other sign conditions on f and f^
Computed the truncated integral
Computed the remainder term of the integral
Verified the integral condition on f^


Systole bound:  161826673/19430721
Decimal approximation:  8.32839260056279


[3.156052254942441,
 4.194718514842724,
 4.87686298531343,
 5.381936511065651,
 5.783670847921718,
 6.117159683790018,
 6.407733695361405,
 6.655634313768748,
 6.880868017761692,
 7.080714268103802,
 7.2627341401966685,
 7.429526630894075,
 7.584858996040415,
 7.729298463371804,
 7.863528481048322,
 7.988772695768114,
 8.118853702218129,
 8.22070946601743,
 8.32839260056279]